In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sys
sys.path.append('../')

from config import *
import pandas as pd
from tools import *
import numpy as np
from sklearn.model_selection import KFold
import lightgbm as lgb
from tqdm import tqdm
from scipy import sparse

In [2]:
### Target #########
TARGET = 'gender'

In [3]:
### target
target = pd.read_pickle(TRAIN_DIR+USER_LOG_PATH)
target = target.groupby(['user_id']).agg('first').reset_index()

# 统计特征

In [6]:
### Per Day Click Times
train_df = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['per_day_click'])
test_df = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['per_day_click'])

In [7]:
assert train_df.shape[0] == 900000 and test_df.shape[0] == 1000000

# 点击 List CounterVec 特征

### creative_id

In [8]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['creative_id'])
ts_click_seq = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['creative_id'])

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['creative_id'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

creative_id_cntv_user = cntv.fit_transform(click_seq)

creative_id_cntv_user = creative_id_cntv_user.astype('float32')

In [9]:
sparse.save_npz(TRAIN_DIR+'creative_id_cntv_user.npz', creative_id_cntv_user)

### ad_id

In [10]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['ad_id'])
ts_click_seq = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['ad_id'])

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['ad_id'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

ad_id_cntv_user = cntv.fit_transform(click_seq)

ad_id_cntv_user = ad_id_cntv_user.astype('float32')

In [11]:
sparse.save_npz(TRAIN_DIR+'ad_id_cntv_user.npz', ad_id_cntv_user)

### advertiser_id

In [12]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['advertiser_id'])
ts_click_seq = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['advertiser_id'])

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['advertiser_id'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

advertiser_id_cntv_user = cntv.fit_transform(click_seq)

advertiser_id_cntv_user = advertiser_id_cntv_user.astype('float32')

In [13]:
sparse.save_npz(TRAIN_DIR+'advertiser_id_cntv_user.npz', advertiser_id_cntv_user)

### product_id

In [14]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['product_id'])
ts_click_seq = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['product_id'])

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['product_id'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

product_id_cntv_user = cntv.fit_transform(click_seq)

product_id_cntv_user = product_id_cntv_user.astype('float32')

In [15]:
sparse.save_npz(TRAIN_DIR+'product_id_cntv_user.npz', product_id_cntv_user)

### product_category

In [16]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['product_category'])
ts_click_seq = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['product_category'])

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['product_category'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

product_category_cntv_user = cntv.fit_transform(click_seq)

product_category_cntv_user = product_category_cntv_user.astype('float32')

In [17]:
sparse.save_npz(TRAIN_DIR+'product_category_cntv_user.npz', product_category_cntv_user)

### industry

In [18]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+CLK_PATH_DICT['industry'])
ts_click_seq = pd.read_pickle(TEST_DIR+CLK_PATH_DICT['industry'])

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['industry'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

industry_cntv_user = cntv.fit_transform(click_seq)

industry_cntv_user = industry_cntv_user.astype('float32')

In [19]:
sparse.save_npz(TRAIN_DIR+'industry_cntv_user.npz', industry_cntv_user)

### Time

In [20]:
tr_click_seq = pd.read_pickle(TRAIN_DIR+'clk_list_time.pkl')
ts_click_seq = pd.read_pickle(TEST_DIR+'clk_list_time.pkl')

assert tr_click_seq['user_id'].values.tolist() == train_df.index.values.tolist()
assert ts_click_seq['user_id'].values.tolist() == test_df.index.values.tolist()

click_seq = pd.concat([tr_click_seq, ts_click_seq], axis=0)['time'].values.tolist()

cntv = CountVectorizer(tokenizer=lambda x: x, max_features=100000, max_df=0.9, min_df=30, lowercase=False)

time_cntv_user = cntv.fit_transform(click_seq)

time_cntv_user = time_cntv_user.astype('float32')

In [21]:
sparse.save_npz(TRAIN_DIR+'time_cntv_user.npz', time_cntv_user)

### Merge Feature

In [ ]:
train_csr = sparse.csr_matrix(train_df.values)
test_csr = sparse.csr_matrix(test_df.values)

In [ ]:
train_csr = sparse.hstack((train_csr, cntv_user[:900000])).tocsr()
test_csr  = sparse.hstack((test_csr , cntv_user[900000:])).tocsr()

In [ ]:
X_train, y_train = train_csr, target['age']-1

In [ ]:
X_test = test_csr

# Metric

In [ ]:
def accuracy(y_true, y_pred):
    assert len(y_true) == len(y_pred), "length of y_true and y_pred not equal"
    total_example = len(y_true)
    right_cnt = 0
    for t, p in zip(y_true, y_pred):
        if t == p:
            right_cnt += 1
    return right_cnt / total_example

# Model

In [ ]:
param = { 
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 10,  
    'metric': ['multi_error'],  
    'num_leaves': 2**9,  
    'min_data_in_leaf': 500,  
    'learning_rate': 0.1,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1,
    'num_threads':6,
    'n_estimators': 1000
}

In [ ]:
N_SPLITS = 5
folds = KFold(n_splits=N_SPLITS, shuffle=True, random_state=np.random.randint(2020))
# 五折交叉验证
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    clf = lgb.train(param, 
                    trn_data, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 100, 
                    early_stopping_rounds = 200)
    
    y_val_pred = clf.predict(X_train[val_idx])
    y_val_pred = np.argmax(y_val_pred,axis=-1).tolist()
    acc = accuracy(y_train[val_idx].values.tolist(), y_val_pred)
    print("kfold: {:d}, accuracy: {:.4f}".format(fold_+1, acc))
    
    break

In [ ]:
clf.save_model('/home/huangzc/competition/tencent/model_ckpt/lgb/model-1.35.txt')